In [42]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import src.graph_ops as go
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torch.utils import data
from sklearn.preprocessing import OneHotEncoder

In [43]:
graphs = np.load("data/graph_10node7500.npy")
# options: graph_10+20k, graph_10+10k
labels = np.load("data/labels_10node7500.npy")
# options: labels_10+20k, labels_10+10k
X = go.compress_graphs(graphs)


In [44]:
y = pd.DataFrame(np.zeros((labels.shape[0], 10)))
y = y.add(pd.get_dummies(labels[:,0]), fill_value=0)
y = y.add(pd.get_dummies(labels[:,1]), fill_value=0)
y = y.values.astype(int)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [46]:
def cross_entropy(input, target, size_average=True):
    return torch.mean(-target * torch.log(input))


class MLP(nn.Module): 
    
    #constructor
    #take in X as a parameter
    def __init__(self, X, y, weight_decay = 0.0005):
        super(MLP, self).__init__()
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #self.device = 'cpu'
        
        y_dim = y.shape[-1]
        X_dim = X.shape[-1]
        
        
        
        self.fc1 = nn.Linear(X_dim, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 50)
        self.fc5 = nn.Linear(50, 50)
        self.fc6 = nn.Linear(50, y_dim)
        self.sigmoid = nn.Sigmoid()

        
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay = weight_decay)
        self.to(self.device)
        
    # 1. input X
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc6(x)
        x = self.sigmoid(x)
        x = 2*x/(x.sum(1)[:,None])
        return x
    
    def loss(self, pred, true):
        #PyTorch's own cross entropy loss function.
        score = cross_entropy
        l = None
        for i in range(true.shape[1]):
            if l is None:
                l = score(pred[:,i], true[:,i])
            else:
                l += score(pred[:,i], true[:,i])
        return l
    

    def fit(self, X, y, early_stopping = True, patience = 50):
        
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y_arr = y.values
            if len(y_arr.shape) == 1:
                y_arr = y_arr[:,None]
            y_tens = Variable(torch.Tensor(y_arr).float())
        else:
            if len(y.shape) == 1:
                y = y[:,None]
            y_tens = Variable(torch.Tensor(y).float())
        
        X_tens = X_tens.to(self.device)
        y_tens = y_tens.to(self.device)
        
        if early_stopping == True:
            
            inds = np.arange(len(X_tens))
            num_train = int(len(X_tens) * .9)
            train_inds = np.random.choice(inds, num_train, replace=False)
            val_inds = np.setdiff1d(inds, train_inds)
            X_train = X_tens[train_inds]
            y_train = y_tens[train_inds]
            X_val = X_tens[val_inds]
            y_val = y_tens[val_inds]
            
        for epoch in range(1000):
            self.optimizer.zero_grad()
            if early_stopping==True:
                out_train = self.forward(X_train)
                loss = self.loss(out_train, y_train)
            if early_stopping==False:
                out_train = self.forward(X_tens)
                loss = self.loss(out_train, y_tens)
                print(loss)
                loss.backward()
                self.optimizer.step()
                continue
                
            out_val = self.forward(X_val)
            loss_val = self.loss(out_val, y_val)
            print(loss_val)


            if epoch == 0:
                min_loss = loss_val
                best_weights = self.state_dict()
                counter = 0
            else:
                if loss_val < min_loss:
                    min_loss = loss_val
                    best_weights = self.state_dict()
                    counter = 0
                else:
                    counter += 1
                    if counter == patience:
                        self.load_state_dict(best_weights)
                        return
                
            loss.backward()
            self.optimizer.step()
    
    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
            
        X_tens = X_tens.to(self.device)
        
        predict_out = self.forward(X_tens)
        temp =  predict_out.cpu().detach().numpy()
        temp[temp > 1-1e-6] = 1-1e-6
        temp[temp < 1e-6]=1e-6
        return temp
    
    def predict(self, X):
        proba = self.predict_proba(X).argsort(1)[:,-2:]
        proba.sort(1)
        return proba
    
    def score(self, X, y):
        y_pred = self.predict(X)
        y_true = y.argsort(1)[:,-2:]
        y_true.sort(1)
        return (y_pred == y_true).all(1).mean().item()
    
    def top_k_acc(self, X,y, k = 5):
        """Get top k accuracy for prediction. I.e., were the 2 correct nodes 
        in the top k returned. Note, k=2 is the score function."""
        y_pred = self.predict_proba(X)
        y_true = y.argsort(1)[:,-2:]  
        y_true.sort(1)
        eqs_list = []
        top_k = y_pred.argsort(1)[:,-k:]
        for i in range(y_true.shape[0]):
            row = y_true[i]
            eqs_arr = []
            for ele in row:
                eqs_arr.append(np.any(ele == top_k[i]))
            eqs_arr = np.array(eqs_arr)
            eq = eqs_arr.all()
            eqs_list.append(eq)
        eqs = np.array(eqs_list)

        return eqs.mean()

In [47]:
mlp = MLP(X, y)

In [48]:
mlp.fit(X_train,y_train)

tensor(3.1996, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1914, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1827, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1735, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1640, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1539, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1430, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1310, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1175, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1026, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0859, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0675, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0472, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0010, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9752, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9478, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9188, device='cuda:0', grad_fn=<AddBack

tensor(1.7508, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7456, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7402, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7342, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7281, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7215, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7148, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7079, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7005, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6931, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6851, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6772, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6686, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6599, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6508, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6414, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6315, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6211, device='cuda:0', grad_fn=<AddBack

tensor(0.9607, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9657, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9369, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9652, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9785, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9449, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9862, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9320, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9570, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9183, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9684, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9139, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9369, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9161, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9382, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9118, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9163, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9019, device='cuda:0', grad_fn=<AddBack

tensor(0.7404, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7561, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7409, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7421, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7402, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7302, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7384, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7292, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7342, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7354, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7311, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7347, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7258, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7261, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7282, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7256, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7309, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7270, device='cuda:0', grad_fn=<AddBack

tensor(0.6833, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6819, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6801, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6805, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6886, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6831, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6846, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6856, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7016, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6881, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6870, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6834, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6893, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6784, device='cuda:0', grad_fn=<AddBack

In [49]:
mlp.score(X_test, y_test)

0.8927243759957515

In [52]:
mlp_acc = [mlp.top_k_acc(X_test, y_test, k = i) for i in range(2,11)]

In [53]:
mlp_acc

[0.8927243759957515,
 0.9330855018587361,
 0.951141795007966,
 0.966542750929368,
 0.9798194370685077,
 0.9856611789697292,
 0.9915029208709506,
 0.9962825278810409,
 1.0]